In [ ]:
import socket
import time
import re
import mytools
import numpy as np

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(('', 1234))
s.settimeout(5)

In [ ]:
s.send(

In [ ]:
data = {}
cycleNum = 0
cycle = True

# UDP attemption
ltime_tm_min_prev = 61;
while cycle:
    try:
        message, address = s.recvfrom(1024)
        message = str(message)
        message = re.sub(r'^(b\')', '', message)
        message = re.sub(r'[\r\n\']', '', message, 0, re.MULTILINE)
        message_list = re.split(';', message)#, maxsplit=0)
        for message in message_list:
            message = re.split('=', message)#, maxsplit=0)
            if len(message) == 2:
                mkey = message[0].strip()
                mval = message[1]
                mval = re.sub(r'[^0-9\.,\-]', '', mval)
                mval = mval.replace(',', '.') 
                mval = float(mval)
                if mkey not in data.keys():
                    data[mkey] = []
                    print('*',sep='',end='')
                else:
                    print('.',sep='',end='')
                    pass
                data[mkey].append(mval)
    except (TimeoutError):
        print('?',sep='',end='')

    ltime = time.localtime()
    if ltime.tm_min in [x for x in range(0, 60, 1)] and ltime.tm_min != ltime_tm_min_prev:
        ltime_tm_min_prev = ltime.tm_min
        data_d = {}
        for key, mas in data.items():
            a = np.array(mas)
            data_d[key] = a.mean()
            data_d[key+'_err'] = a.std()
        data_d['DateTime'] = time.strftime('%d.%m.%Y %H:%M:%S', time.localtime())
        print(data_d)
        link = mytools.connectBase(**sql_param_dict)
        cursor = link.cursor()

        mytools.writeDictToDB(myDict=data_d,
                              cursor=cursor,
                              baseName='baikal',
                              tableName=season)

        link.close()
        time.sleep(1.001)
        data = {}

s.close()

In [ ]:
s.close()